In [1]:
using BenchmarkTools
using JuMP

In [404]:
include("src/staticvars.jl")

sp

In [405]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()
z = sp.Variable{:z}()

z

In [406]:
x^2 + y + x * x

y + 2x^2

In [407]:
p = x^2 + y

y + x^2

In [408]:
p1, p2 = promote(p, x * x)

(y + x^2, x^2)

In [409]:
sp.jointerms(p1.terms, p2.terms)

2-element Array{sp.Term{Int64,sp.Monomial{2,(:x, :y)}},1}:
 y   
 2x^2

In [410]:
p1.terms[1].monomial > p2.terms[1].monomial

true

In [411]:
p1.terms[1].monomial

y

In [412]:
p1 + p2

y + 2x^2

In [413]:
@which p1 + p2

+(p1::sp.Polynomial{T1,V1}, p2::sp.Polynomial{T2,V2}) where {T1, V1, T2, V2} in sp at /home/rdeits/.julia/v0.6/StaticPolynomials/src/staticvars.jl:212

In [414]:
@which p + x * x

+(t1::sp.PolynomialLike, t2::sp.PolynomialLike) in sp at /home/rdeits/.julia/v0.6/StaticPolynomials/src/staticvars.jl:216

In [415]:
p = x^2 * y + y^2 + x

y^2 + x + x^2y

In [416]:
sp.subs(p, x=>1, y=>2)

7

In [417]:
@code_warntype sp.subs(p, x=>1, y=>2)

Variables:
  #self#::sp.#subs
  v::sp.Polynomial{sp.Term{Int64,sp.Monomial{2,(:x, :y)}},Array{sp.Term{Int64,sp.Monomial{2,(:x, :y)}},1}}
  s::Tuple{Pair{sp.Variable{:x},Int64},Pair{sp.Variable{:y},Int64}}

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location /home/rdeits/.julia/v0.6/StaticPolynomials/src/staticvars.jl subs 298
      # meta: location /home/rdeits/.julia/v0.6/StaticPolynomials/src/staticvars.jl # line 301:
      SSAValue(0) = $(Expr(:invoke, MethodInstance for subs_complete(::sp.Polynomial{sp.Term{Int64,sp.Monomial{2,(:x, :y)}},Array{sp.Term{Int64,sp.Monomial{2,(:x, :y)}},1}}, ::Tuple{Pair{sp.Variable{:x},Int64},Pair{sp.Variable{:y},Int64}}), :(sp.subs_complete), :(v), :(s)))
      # meta: pop location
      # meta: pop location
      $(Expr(:inbounds, :pop))
      return SSAValue(0)
  end::Int64


In [418]:
t1 = p.terms[3]
m1 = t1.monomial

x^2y

In [419]:
@code_warntype sp.subs(m1, x=>1, y=>2)

Variables:
  #self#::sp.#subs
  v::sp.Monomial{2,(:x, :y)}
  s::Tuple{Pair{sp.Variable{:x},Int64},Pair{sp.Variable{:y},Int64}}

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location /home/rdeits/.julia/v0.6/StaticPolynomials/src/staticvars.jl subs 266
      SSAValue(1) = $(Expr(:invoke, MethodInstance for power_by_squaring(::Int64, ::Int64), :(Base.power_by_squaring), :((Core.getfield)((Base.getfield)(s, 1)::Pair{sp.Variable{:x},Int64}, :second)::Int64), :((Base.getfield)((Core.getfield)(v, :exponents)::Tuple{Int64,Int64}, 1)::Int64)))
      SSAValue(0) = $(Expr(:invoke, MethodInstance for power_by_squaring(::Int64, ::Int64), :(Base.power_by_squaring), :((Core.getfield)((Base.getfield)(s, 2)::Pair{sp.Variable{:y},Int64}, :second)::Int64), :((Base.getfield)((Core.getfield)(v, :exponents)::Tuple{Int64,Int64}, 2)::Int64)))
      # meta: pop location
      $(Expr(:inbounds, :pop))
      return (Base.mul_int)(SSAValue(1), SSAValue(0))::Int64
  end::Int64


In [420]:
@benchmark sp.subs($p, $x=>1, $y=>2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     17.355 ns (0.00% GC)
  median time:      25.403 ns (0.00% GC)
  mean time:        24.884 ns (0.00% GC)
  maximum time:     104.622 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [422]:
@benchmark sp.subs($p, $x=>4)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  3
  --------------
  minimum time:     47.667 ns (0.00% GC)
  median time:      62.003 ns (0.00% GC)
  mean time:        83.096 ns (24.92% GC)
  maximum time:     2.901 μs (96.25% GC)
  --------------
  samples:          10000
  evals/sample:     989
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [423]:
sp.subs(p, x=>1)

y^2 + 1 + y

In [424]:
p = x^2 + y + x * x + 3 * x * y + x * y^2

y + 3xy + xy^2 + 2x^2

In [425]:
[x^i for i in 0:3]

4-element Array{sp.Monomial{1,(:x,)},1}:
 1  
 x  
 x^2
 x^3

In [426]:
@code_warntype monomials(x, 3)

Variables:
  #self#::#monomials
  var::sp.Variable{:x}
  degree::Int64
  #13::##13#14{sp.Variable{:x}}

Body:
  begin 
      SSAValue(1) = $(Expr(:new, :($(QuoteNode(UnitRange{Int64}))), 0, :((Base.select_value)((Base.sle_int)(0, degree)::Bool, degree, (Base.sub_int)(0, 1)::Int64)::Int64)))
      SSAValue(2) = $(Expr(:new, :($(QuoteNode(Base.Generator{UnitRange{Int64},##13#14{sp.Variable{:x}}}))), :($(QuoteNode(#13))), SSAValue(1)))
      return $(Expr(:invoke, MethodInstance for collect(::Base.Generator{UnitRange{Int64},##13#14{sp.Variable{:x}}}), :(Base.collect), SSAValue(2)))
  end::Array{sp.Monomial{1,(:x,)},1}


In [427]:
@benchmark monomials($x, 3)

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  1
  --------------
  minimum time:     27.655 ns (0.00% GC)
  median time:      45.063 ns (0.00% GC)
  mean time:        56.239 ns (18.62% GC)
  maximum time:     2.676 μs (94.49% GC)
  --------------
  samples:          10000
  evals/sample:     995
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [428]:
@benchmark $x^2 + $y + $x * $x + 3 * $x * $y + $x * $y^2

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  8
  --------------
  minimum time:     201.522 ns (0.00% GC)
  median time:      225.298 ns (0.00% GC)
  mean time:        294.069 ns (22.10% GC)
  maximum time:     3.873 μs (91.58% GC)
  --------------
  samples:          10000
  evals/sample:     611
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [443]:
@benchmark $x^2 + $y

BenchmarkTools.Trial: 
  memory estimate:  208 bytes
  allocs estimate:  4
  --------------
  minimum time:     56.595 ns (0.00% GC)
  median time:      70.054 ns (0.00% GC)
  mean time:        100.234 ns (28.55% GC)
  maximum time:     4.325 μs (97.11% GC)
  --------------
  samples:          10000
  evals/sample:     986
  time tolerance:   5.00%
  memory tolerance: 1.00%